In [40]:
# Import Libraries,
import pandas as pd
import re

In [41]:
# Read Data,
path = 'Data/flipkart.txt'
f = open(path, 'r', encoding='utf-8')
txt = f.read()

In [42]:
def fileReader(fileName, msgs, start=0):
    try:
        print("\n".join(fileName.split('\n')[start:msgs]))
    except:
        print("\n".join(fileName[start:msgs]))

fileReader(txt, 6)

09/10/21, 2:34 pm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.
09/10/21, 2:34 pm - This chat is with the official business account of Flipkart. Tap to learn more.
09/10/21, 2:34 pm - Flipkart: <Media omitted>
09/10/21, 2:34 pm - This chat is with the official business account of Flipkart. Tap to learn more.
12/11/21, 11:07 am - This business is now working with other companies to manage this chat. Tap to learn more.
18/12/21, 10:10 pm - Flipkart: <Media omitted>


In [43]:
# Seperate the messages and dates,
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[a-zA-Z]+\s-\s'
texts = re.split(pattern, txt)[1:]

In [44]:
fileReader(texts, 4)

Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.

This chat is with the official business account of Flipkart. Tap to learn more.

Flipkart: <Media omitted>

This chat is with the official business account of Flipkart. Tap to learn more.



In [45]:
dates = re.findall(pattern, txt)

In [46]:
fileReader(dates, 6)

09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
12/11/21, 11:07 am - 
18/12/21, 10:10 pm - 


In [47]:
len(texts)

11

In [48]:
# Put everything into a data frame.
dic = {
    'messages' : texts,
    'date' : dates
}

df = pd.DataFrame(dic)
df.sample(3)

,messages,date
0,Messages and calls are end-to-end encrypted. N...,"09/10/21, 2:34 pm -"
4,This business is now working with other compan...,"12/11/21, 11:07 am -"
6,.: Menu\n,"20/12/21, 5:27 pm -"


In [49]:
# Change the date format to datetime format from a string,
df.date[0]

'09/10/21, 2:34\u202fpm - '

In [50]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y, %I:%M %p - ')
df.sample()

,messages,date
8,Flipkart: <Media omitted>\n,2022-06-12 14:00:00


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   messages  11 non-null     object        
 1   date      11 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 304.0+ bytes


In [52]:
df.shape

(11, 2)

In [53]:
def dateConversion(df):

    df['Hour'] = df.date.dt.hour
    df['Minute'] = df.date.dt.minute
    df['Day'] = df.date.dt.day
    df['Month'] = df.date.dt.month_name()
    df['Year'] = df.date.dt.year

    return df.sample(5)

dateConversion(df)


,messages,date,Hour,Minute,Day,Month,Year
8,Flipkart: <Media omitted>\n,2022-06-12 14:00:00,14,0,12,June,2022
6,.: Menu\n,2021-12-20 17:27:00,17,27,20,December,2021
10,This business is now working with other compan...,2023-12-10 01:16:00,1,16,10,December,2023
4,This business is now working with other compan...,2021-11-12 11:07:00,11,7,12,November,2021
9,This business is now working with other compan...,2023-11-19 20:11:00,20,11,19,November,2023


In [55]:
def userSeperator(df):
    '''
    As the messages are in this format: {username}: {His/her message}
    Use regex to match and seperate user and message.
    '''
    usernames = []
    messages = []
    media = []
    pattern = '([\w\W]+?):\s'

    for chat in df['messages']:
        
        content = re.split(pattern, chat)

        if content[1:]:      # beginning from the first index as content contains blank ('') string at 0th index.
            usernames.append(content[1])
            pt = "<Media omitted>"
            st = content[2]
            msg = re.split(pt, st)

            if len(msg)==1:
                media.append(0)
                messages.append(msg[0])

            else:
                media.append(1)
                messages.append(msg[1])

        else:
            usernames.append('Zuckerberg')
            media.append(0)
            messages.append(content[0])
        
    df['message'] = messages
    df['user'] = usernames
    df['media'] = media
    
    return df.sample(5)

userSeperator(df)

,messages,date,Hour,Minute,Day,Month,Year,message,user,media
2,Flipkart: <Media omitted>\n,2021-10-09 14:34:00,14,34,9,October,2021,\n,Flipkart,1
1,This chat is with the official business accoun...,2021-10-09 14:34:00,14,34,9,October,2021,This chat is with the official business accoun...,Zuckerberg,0
8,Flipkart: <Media omitted>\n,2022-06-12 14:00:00,14,0,12,June,2022,\n,Flipkart,1
9,This business is now working with other compan...,2023-11-19 20:11:00,20,11,19,November,2023,This business is now working with other compan...,Zuckerberg,0
6,.: Menu\n,2021-12-20 17:27:00,17,27,20,December,2021,Menu\n,.,0


In [56]:
# Drop the unncessary columns,
df.drop(columns=['date','messages'], inplace=True)
df.sample(2)

,Hour,Minute,Day,Month,Year,message,user,media
9,20,11,19,November,2023,This business is now working with other compan...,Zuckerberg,0
10,1,16,10,December,2023,This business is now working with other compan...,Zuckerberg,0
